<a href="https://colab.research.google.com/github/Droppicode/classify_transactions/blob/main/classify_transactions_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

df = pd.read_excel('categorias_ai.xlsx')

df

,Descricao,Categorias
0,ATACADAO 0632 AS,Supermercado/Alimentação
1,ATACADAO 0650 AS,Supermercado/Alimentação
2,AUTO BAN JUNDIAI,Transporte/Combustível
3,AUTO POSTO CAMPINAS,Transporte/Combustível
4,AUTO POSTO GLOBAL DE,Transporte/Combustível
...,...,...
956,MCDONALDS,Restaurante/Alimentação
957,BOLETO 1031 SICOOB,Pagamento de Fatura/Boleto
958,DROGA 9190 RAIA,Farmácia/Saúde
959,PST BR PETROBRAS 6305,Transporte/Combustível


In [138]:
df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{3,}\b').str.join(' ').str.upper().str.strip()
df['Categorias'] = df['Categorias'].str.upper().str.strip()

categorias = [[u for u in cat.split('/')] for cat in df['Categorias']]
categorias = list(set([i for sub in categorias for i in sub]))

for cat in categorias:
    df[cat] = df['Categorias'].str.contains(cat).astype(int)

df

/tmp/ipython-input-3541829250.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[cat] = df['Categorias'].str.contains(cat).astype(int)


,Descricao,Categorias,BOLETO,ÁGUA,LOJAS DE DEPARTAMENTO,ROUPAS,RECARGA CELULAR,INVESTIMENTOS,GÁS,CONTAS RESIDENCIAIS (ÁGUA),LAZER,TAXAS,RESTAURANTE,ESPORTES,SAÚDE,SERVIÇOS,ALIMENTAÇÃO,ESTÉTICA,RELIGIÃO,EDUCAÇÃO,COSMÉTICO,COMPRAS,PAGAMENTO DE FATURA,CASA,COMBUSTÍVEL,CONTAS RESIDENCIAIS,PAPELARIA,RENDIMENTOS,TRANSPORTE,CAPITALIZAÇÃO,SUPERMERCADO,ALUGUEL,VIAGEM,IMPOSTOS,FARMÁCIA,CONSTRUÇÃO,ÓPTICA,PLANO DE SAÚDE,DOAÇÃO,PETS,LUZ
0,ATACADAO,SUPERMERCADO/ALIMENTAÇÃO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,ATACADAO,SUPERMERCADO/ALIMENTAÇÃO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,AUTO BAN JUNDIAI,TRANSPORTE/COMBUSTÍVEL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,AUTO POSTO CAMPINAS,TRANSPORTE/COMBUSTÍVEL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,AUTO POSTO GLOBAL,TRANSPORTE/COMBUSTÍVEL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,MCDONALDS,RESTAURANTE/ALIMENTAÇÃO,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
957,BOLETO SICOOB,PAGAMENTO DE FATURA/BOLETO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
958,DROGA RAIA,FARMÁCIA/SAÚDE,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
959,PST PETROBRAS,TRANSPORTE/COMBUSTÍVEL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

X = df['Descricao']
y = df[categorias]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

model = make_pipeline(
    TfidfVectorizer(),
    MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
)

print("Treinando modelos para todas as colunas...")
model.fit(X_train, y_train)
print("Treino concluído!")

Treinando modelos para todas as colunas...
Treino concluído!


In [140]:
lista_probas = model.predict_proba(X_test)

probs_limpos = []
for proba in lista_probas:
    if proba.shape[1] == 2: probs_limpos.append(proba[:, 1])
    else: probs_limpos.append(np.zeros(proba.shape[0]))

matriz_probas = np.array(probs_limpos).T

df_final = pd.DataFrame(matriz_probas, columns=categorias, index=X_test.index)
df_final.insert(0, 'Descricao', df.loc[X_test.index, 'Descricao'])
df_final.insert(1, 'Categorias', df.loc[X_test.index, 'Categorias'])
df_final.insert(2, 'Predicted', None)
df_final = df_final.sort_index()

for index, row in df_final.iterrows():
  pred = [cat for cat in categorias if row[cat] > 0.5]
  df_final.at[index, 'Predicted'] = pred

df_final

,Descricao,Categorias,Predicted,BOLETO,ÁGUA,LOJAS DE DEPARTAMENTO,ROUPAS,RECARGA CELULAR,INVESTIMENTOS,GÁS,CONTAS RESIDENCIAIS (ÁGUA),LAZER,TAXAS,RESTAURANTE,ESPORTES,SAÚDE,SERVIÇOS,ALIMENTAÇÃO,ESTÉTICA,RELIGIÃO,EDUCAÇÃO,COSMÉTICO,COMPRAS,PAGAMENTO DE FATURA,CASA,COMBUSTÍVEL,CONTAS RESIDENCIAIS,PAPELARIA,RENDIMENTOS,TRANSPORTE,CAPITALIZAÇÃO,SUPERMERCADO,ALUGUEL,VIAGEM,IMPOSTOS,FARMÁCIA,CONSTRUÇÃO,ÓPTICA,PLANO DE SAÚDE,DOAÇÃO,PETS,LUZ
0,ATACADAO,SUPERMERCADO/ALIMENTAÇÃO,"[ROUPAS, COMPRAS]",0.00,0.0,0.0,0.84,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.007182,0.0,0.0,0.0,0.84,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.00
1,ATACADAO,SUPERMERCADO/ALIMENTAÇÃO,"[ROUPAS, COMPRAS]",0.00,0.0,0.0,0.84,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.007182,0.0,0.0,0.0,0.84,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.00
3,AUTO POSTO CAMPINAS,TRANSPORTE/COMBUSTÍVEL,"[COMBUSTÍVEL, TRANSPORTE]",0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.01,0.00,0.0,0.96,0.0,0.0,0.0,0.95,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
14,COMERCIODE,SUPERMERCADO/ALIMENTAÇÃO,"[ROUPAS, COMPRAS]",0.00,0.0,0.0,0.84,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.007182,0.0,0.0,0.0,0.84,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.00
15,CONDE RESTAURANTE,RESTAURANTE/ALIMENTAÇÃO,[],0.00,0.0,0.0,0.35,0.0,0.0,0.0,0.0,0.0,0.00,0.48,0.00,0.0,0.00,0.46,0.007182,0.0,0.0,0.0,0.42,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,NETSHOES N73,ROUPAS/COMPRAS/ESPORTES,"[ROUPAS, ESPORTES, COMPRAS]",0.00,0.0,0.0,0.95,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.86,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.95,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
940,CTA AGUA,CONTAS RESIDENCIAIS/ÁGUA,"[ÁGUA, CONTAS RESIDENCIAIS]",0.00,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.00,1.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
941,PETZ,PETS,[PETS],0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.00,0.00
944,PGTO BRADESCO,PAGAMENTO DE FATURA/BOLETO,"[BOLETO, PAGAMENTO DE FATURA]",0.93,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.01,0.00,0.000000,0.0,0.0,0.0,0.00,0.93,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.03,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.01


In [143]:
model.fit(X, y)

df = pd.read_excel('teste.xlsx')
df = df[~df['Descricao'].str.contains(r'\bPIX\b', regex=True)]

lista_probas = model.predict_proba(df['Descricao'])

probs_limpos = []
for proba in lista_probas:
    if proba.shape[1] == 2: probs_limpos.append(proba[:, 1])
    else: probs_limpos.append(np.zeros(proba.shape[0]))

matriz_probas = np.array(probs_limpos).T

df_final = pd.DataFrame(matriz_probas, columns=categorias, index=df.index)
df_final.insert(0, 'Descricao', df.loc[df.index, 'Descricao'])
df_final.insert(1, 'Categorias', df.loc[df.index, 'Categoria'])
df_final.insert(2, 'Predicted', None)
df_final = df_final.sort_index()

for index, row in df_final.iterrows():
  pred = [cat for cat in categorias if row[cat] > 0.5]
  df_final.at[index, 'Predicted'] = pred

df_final

,Descricao,Categorias,Predicted,BOLETO,ÁGUA,LOJAS DE DEPARTAMENTO,ROUPAS,RECARGA CELULAR,INVESTIMENTOS,GÁS,CONTAS RESIDENCIAIS (ÁGUA),LAZER,TAXAS,RESTAURANTE,ESPORTES,SAÚDE,SERVIÇOS,ALIMENTAÇÃO,ESTÉTICA,RELIGIÃO,EDUCAÇÃO,COSMÉTICO,COMPRAS,PAGAMENTO DE FATURA,CASA,COMBUSTÍVEL,CONTAS RESIDENCIAIS,PAPELARIA,RENDIMENTOS,TRANSPORTE,CAPITALIZAÇÃO,SUPERMERCADO,ALUGUEL,VIAGEM,IMPOSTOS,FARMÁCIA,CONSTRUÇÃO,ÓPTICA,PLANO DE SAÚDE,DOAÇÃO,PETS,LUZ
0,PADARIA MONACO,SUPERMERCADO/ALIMENTAÇÃO,"[RESTAURANTE, ALIMENTAÇÃO]",0.0,0.00,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.78,0.0,0.00,0.00,0.89,0.0,0.0,0.0,0.00,0.05,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
3,MED FARMA,SAÚDE/FARMÁCIA,"[SAÚDE, FARMÁCIA]",0.0,0.00,0.0,0.45,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.63,0.00,0.00,0.0,0.0,0.0,0.00,0.44,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.66,0.00,0.00,0.0,0.0,0.0,0.0
4,MEUMARKET24H,SUPERMERCADO/ALIMENTAÇÃO,"[ALIMENTAÇÃO, SUPERMERCADO]",0.0,0.00,0.0,0.17,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.82,0.0,0.0,0.0,0.00,0.17,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.82,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
5,EMPORIO VILA NATURAL,SUPERMERCADO/ALIMENTAÇÃO,"[ALIMENTAÇÃO, SUPERMERCADO]",0.0,0.00,0.0,0.21,0.0,0.0,0.12,0.0,0.0,0.00,0.03,0.0,0.00,0.00,0.72,0.0,0.0,0.0,0.00,0.22,0.0,0.00,0.0,0.10,0.0,0.0,0.0,0.0,0.68,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
6,MEUMARKET24H,SUPERMERCADO/ALIMENTAÇÃO,"[ALIMENTAÇÃO, SUPERMERCADO]",0.0,0.00,0.0,0.17,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.82,0.0,0.0,0.0,0.00,0.17,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.82,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,ASSOCIACÃO INSTITUTO,EDUCAÇÃO/DOAÇÃO,"[ALIMENTAÇÃO, SUPERMERCADO]",0.0,0.00,0.0,0.12,0.0,0.0,0.00,0.0,0.0,0.01,0.06,0.0,0.00,0.01,0.85,0.0,0.0,0.0,0.00,0.09,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.71,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
125,SABESP,CONTAS RESIDENCIAIS/ÁGUA,"[ÁGUA, ROUPAS, COMPRAS]",0.0,0.53,0.0,0.65,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.63,0.0,0.00,0.0,0.44,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
126,GOOD BOM,SUPERMERCADO/ALIMENTAÇÃO,"[ALIMENTAÇÃO, SUPERMERCADO]",0.0,0.00,0.0,0.11,0.0,0.0,0.00,0.0,0.0,0.00,0.01,0.0,0.26,0.00,0.71,0.0,0.0,0.0,0.29,0.15,0.0,0.03,0.0,0.00,0.0,0.0,0.0,0.0,0.64,0.0,0.0,0.0,0.03,0.03,0.29,0.0,0.0,0.0,0.0
128,GOOD BOM,SUPERMERCADO/ALIMENTAÇÃO,"[ALIMENTAÇÃO, SUPERMERCADO]",0.0,0.00,0.0,0.11,0.0,0.0,0.00,0.0,0.0,0.00,0.01,0.0,0.26,0.00,0.71,0.0,0.0,0.0,0.29,0.15,0.0,0.03,0.0,0.00,0.0,0.0,0.0,0.0,0.64,0.0,0.0,0.0,0.03,0.03,0.29,0.0,0.0,0.0,0.0
